# English to Spanish - Machine Translation - Sequence to sequence - RNN

In [3]:
# English to spanish data base
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2022-06-20 15:03:38--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.01s   

2022-06-20 15:03:38 (232 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [4]:
# parse the file
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]
text_pairs = [] 
for line in lines:
  english, spanish = line.split("\t")
  spanish = "[start] " + spanish + " [end]"
  text_pairs.append((english, spanish))

In [5]:
# random data
import random
print(random.choice(text_pairs))

('I want you to talk to Tom.', '[start] Quiero que hables con Tom. [end]')


In [6]:
# Shuffle and split
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [8]:
# Listing 11.26 Vectorizing the English and Spanish text pairs
import tensorflow as tf 
import string
import re
from tensorflow.keras import layers
 
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
      lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [9]:
# Listing 11.27 Preparing datasets for the translation task
batch_size = 64
 
def format_dataset(eng, spa):
  eng = source_vectorization(eng)
  spa = target_vectorization(spa)
  return ({
      "english": eng,
      "spanish": spa[:, :-1],
      }, 
      spa[:, 1:]) 
  
def make_dataset(pairs):
  eng_texts, spa_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  spa_texts = list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=1)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [10]:
# dataset outputs look like
for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape: {inputs['english'].shape}")
  print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
  print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [11]:
# Listing 11.28 GRU-based encoder
from tensorflow import keras 
from tensorflow.keras import layers
 
embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

In [12]:
# Listing 11.29 GRU-based decoder and the end-to-end model
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step) 

In [13]:
seq2seq_rnn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    3840000     ['english[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    3840000     ['spanish[0][0]']                
                                                                                              

In [15]:
# Listing 11.30 Training our recurrent sequence-to-sequence model
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

seq2seq_rnn.fit(train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5
1302/1302 [==============================] - 134s 95ms/step - loss: 1.3238 - accuracy: 0.5257 - val_loss: 1.1519 - val_accuracy: 0.5683
Epoch 2/5
1302/1302 [==============================] - 120s 92ms/step - loss: 1.1777 - accuracy: 0.5757 - val_loss: 1.0752 - val_accuracy: 0.5984
Epoch 3/5
1302/1302 [==============================] - 119s 91ms/step - loss: 1.0858 - accuracy: 0.6077 - val_loss: 1.0350 - val_accuracy: 0.6179
Epoch 4/5
1302/1302 [==============================] - 119s 91ms/step - loss: 1.0377 - accuracy: 0.6323 - val_loss: 1.0208 - val_accuracy: 0.6293
Epoch 5/5
1302/1302 [==============================] - 119s 91ms/step - loss: 1.0074 - accuracy: 0.6506 - val_loss: 1.0171 - val_accuracy: 0.6355


In [16]:
# Listing 11.31 Translating new sentences with our RNN encoder and decoder
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
  tokenized_input_sentence = source_vectorization([input_sentence])
  decoded_sentence = "[start]" 
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = target_vectorization([decoded_sentence])
    next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs] 
for _ in range(20):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))

-
I've corrected the mistake.
[start] he [UNK] el error [end]
-
Something tells me Tom will be OK.
[start] algo que me va a tom se está bien [end]
-
He saw the boy jump over the fence and run away.
[start] Él vio a la [UNK] el [UNK] y se [UNK] a la [UNK] [end]
-
Why do the five yen coin and the fifty yen coin have holes in the center?
[start] por qué los [UNK] la [UNK] y el [UNK] se [UNK] en el cinco de la [UNK] [end]
-
Tom visited Mary on October 20th.
[start] tom le pasó a mary en el [UNK] de [UNK] de [UNK] [end]
-
We are teachers.
[start] estamos [UNK] [end]
-
The explosion may have been caused by a gas leak.
[start] la que se puede haber [UNK] un [UNK] de [UNK] [end]
-
I'm at the beach.
[start] estoy en la playa [end]
-
Tom knows Mary won't tell John.
[start] tom sabe que mary no se lo [UNK] a john [end]
-
Do you like to be alone?
[start] te gusta estar solo [end]
-
Tell me what I should be watching for.
[start] dime qué debería estar en punto [end]
-
What grade are you in?
[start]

---